In [ ]:
# Dependencies required 
!pip install -q -U "tensorflow-text==2.11.*"
import re
import time
import pathlib

import numpy as np
import matplotlib.pyplot as plt

import tensorflow_text as text
import tensorflow as tf
tf.get_logger().setLevel('ERROR')

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 41.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 588.3/588.3 MB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 96.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 77.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 99.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 439.2/439.2 kB 47.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.9/4.9 MB 85.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 781.3/781.3 kB 71.9 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-datasets 4.9.3 requires protobuf>=3.20, but you have protobuf 3.19.6 which is incompatible.
tensorflow-metadata 1.14.0 requires protobuf<4.21,>=3.20.3, but you have protob

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split

In [ ]:
# Data loading and preprocessing
df = pd.read_csv('./data/final.csv')
df = df.drop(['Unnamed: 0','id'], axis=1)

train, test = train_test_split(df, test_size=0.1, random_state=42)

test_ds = tf.data.Dataset.from_tensor_slices(test)
train_ds = tf.data.Dataset.from_tensor_slices(train)

def map_func(x):
  x1 = x[0]
  x2 = x[1]
  return (x1,x2,)
train_ds = train_ds.map(map_func)
test_ds = test_ds.map(map_func)

Tensor("strided_slice:0", shape=(), dtype=string) Tensor("strided_slice_1:0", shape=(), dtype=string)
Tensor("strided_slice:0", shape=(), dtype=string) Tensor("strided_slice_1:0", shape=(), dtype=string)


In [ ]:
train_en = train_ds.map(lambda shk, en: en)
train_shk = train_ds.map(lambda shk, en: shk)

In [ ]:
from tensorflow_text.tools.wordpiece_vocab import bert_vocab_from_dataset as bert_vocab

In [ ]:
bert_tokenizer_params=dict(lower_case=True)
reserved_tokens=["[PAD]", "[UNK]", "[START]", "[END]"]

# Arguments passed when initializing a BertTokenizer
bert_vocab_args = dict(
    # The maximum vocabulary size
    vocab_size = 80_000,
    # Reserved tokens that must be included in the vocabulary
    reserved_tokens=reserved_tokens,
    # Arguments for `text.BertTokenizer`
    bert_tokenizer_params=bert_tokenizer_params,
    # Arguments for `wordpiece_vocab.wordpiece_tokenizer_learner_lib.learn`
    learn_params={},
)


In [ ]:
# Generate vocabulary of the tokenizer for shakespeare dataset.
%%time
shk_vocab = bert_vocab.bert_vocab_from_dataset(
    train_shk.batch(1000).prefetch(2),
    **bert_vocab_args
)

CPU times: user 1min 10s, sys: 712 ms, total: 1min 10s
Wall time: 1min 13s


In [ ]:
# Check the vocabulary
print(shk_vocab[:10])
print(shk_vocab[100:110])
print(shk_vocab[1000:1010])
print(shk_vocab[-10:])

['[PAD]', '[UNK]', '[START]', '[END]', '!', '"', '&', "'", '(', ')']
['from', 'sir', '##ed', 'come', 'll', 'she', 'at', 'they', 'or', 'which']
['##ge', 'choose', 'commend', 'judge', 'laugh', 'order', 'patient', 'roman', 'wild', 'year']
['##]', '##j', '##q', '##æ', '##–', '##—', '##‘', '##’', '##“', '##”']


In [ ]:
# Helper function to save the vocab object to a file
def write_vocab_file(filepath, vocab):
  with open(filepath, 'w') as f:
    for token in vocab:
      print(token, file=f)


In [ ]:
# Save the vocabulary to a shk_vocab.txt
write_vocab_file('./data/shk_vocab.txt', shk_vocab)

In [ ]:
# Generate an english vocabulary for BertTokenzier
%%time
en_vocab = bert_vocab.bert_vocab_from_dataset(
    train_en.batch(1000).prefetch(2),
    **bert_vocab_args
)

CPU times: user 45.8 s, sys: 840 ms, total: 46.6 s
Wall time: 46.7 s


In [ ]:
# Check the vocabulary
print(en_vocab[:10])
print(en_vocab[100:110])
print(en_vocab[1000:1010])
print(en_vocab[-10:])

['[PAD]', '[UNK]', '[START]', '[END]', '!', '"', '$', "'", '(', ')']
['by', 'no', 'now', 'good', 'would', 'she', 'here', 'when', 'don', 'then']
['rude', 'takes', 'wedding', '##man', 'choose', 'earl', 'enjoy', 'friar', 'heads', 'learn']
['##]', '##j', '##q', '##v', '##–', '##—', '##‘', '##’', '##“', '##”']


In [ ]:
# Save the vocabulary to en_vocab.txt
write_vocab_file('./data/en_vocab.txt', en_vocab)

In [ ]:
START = tf.argmax(tf.constant(reserved_tokens) == "[START]")
END = tf.argmax(tf.constant(reserved_tokens) == "[END]")

def add_start_end(ragged):
  count = ragged.bounding_shape()[0]
  starts = tf.fill([count,1], START)
  ends = tf.fill([count,1], END)
  return tf.concat([starts, ragged, ends], axis=1)


In [ ]:
def cleanup_text(reserved_tokens, token_txt):
  # Drop the reserved tokens, except for "[UNK]".
  bad_tokens = [re.escape(tok) for tok in reserved_tokens if tok != "[UNK]"]
  bad_token_re = "|".join(bad_tokens)

  bad_cells = tf.strings.regex_full_match(token_txt, bad_token_re)
  result = tf.ragged.boolean_mask(token_txt, ~bad_cells)

  # Join them into strings.
  result = tf.strings.reduce_join(result, separator=' ', axis=-1)

  return result

In [ ]:
# Create a tokenizer class that includes methods to tokenize and detokenize sentences
class CustomTokenizer(tf.Module):
  def __init__(self, reserved_tokens, vocab_path):
    self.tokenizer = text.BertTokenizer(vocab_path, lower_case=True)
    self._reserved_tokens = reserved_tokens
    self._vocab_path = tf.saved_model.Asset(vocab_path)

    vocab = pathlib.Path(vocab_path).read_text().splitlines()
    self.vocab = tf.Variable(vocab)

    ## Create the signatures for export:

    # Include a tokenize signature for a batch of strings.
    self.tokenize.get_concrete_function(
        tf.TensorSpec(shape=[None], dtype=tf.string))

    # Include `detokenize` and `lookup` signatures for:
    #   * `Tensors` with shapes [tokens] and [batch, tokens]
    #   * `RaggedTensors` with shape [batch, tokens]
    self.detokenize.get_concrete_function(
        tf.TensorSpec(shape=[None, None], dtype=tf.int64))
    self.detokenize.get_concrete_function(
          tf.RaggedTensorSpec(shape=[None, None], dtype=tf.int64))

    self.lookup.get_concrete_function(
        tf.TensorSpec(shape=[None, None], dtype=tf.int64))
    self.lookup.get_concrete_function(
          tf.RaggedTensorSpec(shape=[None, None], dtype=tf.int64))

    # These `get_*` methods take no arguments
    self.get_vocab_size.get_concrete_function()
    self.get_vocab_path.get_concrete_function()
    self.get_reserved_tokens.get_concrete_function()

  @tf.function
  def tokenize(self, strings):
    enc = self.tokenizer.tokenize(strings)
    # Merge the `word` and `word-piece` axes.
    enc = enc.merge_dims(-2,-1)
    enc = add_start_end(enc)
    return enc

  @tf.function
  def detokenize(self, tokenized):
    words = self.tokenizer.detokenize(tokenized)
    return cleanup_text(self._reserved_tokens, words)

  @tf.function
  def lookup(self, token_ids):
    return tf.gather(self.vocab, token_ids)

  @tf.function
  def get_vocab_size(self):
    return tf.shape(self.vocab)[0]

  @tf.function
  def get_vocab_path(self):
    return self._vocab_path

  @tf.function
  def get_reserved_tokens(self):
    return tf.constant(self._reserved_tokens)


In [ ]:
# Create two such tokenizer to convert between the two datasets.
tokenizers = tf.Module()
tokenizers.shk = CustomTokenizer(reserved_tokens, 'shk_vocab.txt')
tokenizers.en = CustomTokenizer(reserved_tokens, 'en_vocab.txt')

In [ ]:
# Export the tokenizer to be used in the transformer notebook.
save_path = './weights/shk_to_en_tokenizers'
tf.saved_model.save(tokenizers, save_path)